## Analysis:

In [19]:
new_column_names = ["PostalCode", "Borough", "Neighborhood", "Latitude", "Longitude"]
newtest_df = pd.DataFrame(columns=new_column_names)

In [20]:
test_list = ["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]

In [24]:
for postcode in test_list:
    newtest_df = newtest_df.append(new_df_toronto[new_df_toronto["PostalCode"]==postcode], ignore_index=True)
    
newtest_df

address = 'Toronto'


geolocator = Nominatim(user_agent="http")

location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


#### Use Folium to create a map of Toronto using the latitude and longitude values & Add Markers:

In [29]:
toronto_map = folium.Map(location=[latitude, longitude], zoom_start=10)

#Add Markers:
for lat, lng, borough, neighborhood in zip(new_df_toronto['Latitude'], new_df_toronto['Longitude'], new_df_toronto['Borough'], new_df_toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(toronto_map)  

toronto_map

#### Filter the Borough names that contain the word 'Toronto':

In [31]:
borough_names = list(new_df_toronto.Borough.unique())

borough_w_toronto = []

for x in borough_names:
    if "toronto" in x.lower():
        borough_w_toronto.append(x)
        
borough_w_toronto

['East Toronto', 'Central Toronto', 'Downtown Toronto', 'West Toronto']

#### Create a DF from it:

In [32]:
new_df_toronto = new_df_toronto[new_df_toronto['Borough'].isin(borough_w_toronto)].reset_index(drop=True)
print(new_df_toronto.shape)
new_df_toronto.head()

(39, 5)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [33]:
# create map of Toronto using lat. and long. values
toronto_map = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers:
for lat, lng, borough, neighborhood in zip(new_df_toronto['Latitude'], new_df_toronto['Longitude'], new_df_toronto['Borough'], new_df_toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(toronto_map)  
    
toronto_map

#### Foursquare Credentials;

In [34]:
CLIENT_ID = 'GZ4I3I1HNV21DWL4MFWPBSYQJZH4DUSF2K1JEKV1V5N3LILK' # your Foursquare ID
CLIENT_SECRET = 'NRSVNZU4LQDJA0N11A0HQIQEAXEAKMDZPUZSUQRSRZJVPDUK' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID:CLIENT_SECRET:

In [35]:
radius = 500
LIMIT = 100

venues = []

for lat, long, post, borough, neighborhood in zip(new_df_toronto['Latitude'], new_df_toronto['Longitude'], new_df_toronto['PostalCode'], 
                                                  new_df_toronto['Borough'], new_df_toronto['Neighborhood']):
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    for venue in results:
        venues.append((
            post, 
            borough,
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

#### Converting the list of Venues into a new DF:

In [38]:
df_venues = pd.DataFrame(venues)

#### Defining the column names

In [40]:
df_venues.columns = ['PostalCode', 'Borough', 'Neighborhood', 'BoroughLatitude', 'BoroughLongitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(df_venues.shape)
df_venues.head()

df_venues.groupby(["PostalCode", "Borough", "Neighborhood"]).count()

print('There are {} uniques categories.'.format(len(df_venues['VenueCategory'].unique())))

df_venues['VenueCategory'].unique()[:50]

(1620, 9)
There are 229 uniques categories.


array(['Trail', 'Health Food Store', 'Pub', 'Neighborhood',
       'Cosmetics Shop', 'Greek Restaurant', 'Ice Cream Shop',
       'Italian Restaurant', 'Brewery', 'Juice Bar', 'Yoga Studio',
       'Restaurant', 'Fruit & Vegetable Store', 'Pizza Place',
       'Bookstore', 'Dessert Shop', 'Furniture / Home Store',
       'Bubble Tea Shop', 'Spa', 'Café', 'Grocery Store', 'Bakery',
       'Coffee Shop', 'Caribbean Restaurant', 'Indian Restaurant',
       'Lounge', 'Frozen Yogurt Shop', 'Sushi Restaurant', 'Liquor Store',
       'American Restaurant', 'Gym', 'Fish & Chips Shop',
       'Fast Food Restaurant', 'Park', 'Burrito Place', 'Pet Store',
       'Steakhouse', 'Movie Theater', 'Sandwich Place', 'Fish Market',
       'Gay Bar', 'Cheese Shop', 'Seafood Restaurant',
       'Middle Eastern Restaurant', 'Stationery Store',
       'Comfort Food Restaurant', 'Thai Restaurant', 'Coworking Space',
       'Wine Bar', 'Latin American Restaurant'], dtype=object)

#### Using One Hot Encoding:

In [41]:
toronto_onehot = pd.get_dummies(df_venues[['VenueCategory']], prefix="", prefix_sep="")

#### Adding the 3 Original Columns to DF:

In [42]:
toronto_onehot['PostalCode'] = df_venues['PostalCode'] 
toronto_onehot['Borough'] = df_venues['Borough'] 
toronto_onehot['Neighborhoods'] = df_venues['Neighborhood'] 

In [43]:
fixed_columns = list(toronto_onehot.columns[-3:]) + list(toronto_onehot.columns[:-3])
toronto_onehot = toronto_onehot[fixed_columns]

In [44]:
print(toronto_onehot.shape)
toronto_onehot.head()

(1620, 232)


,PostalCode,Borough,Neighborhoods,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hospital,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,M4E,East Toronto,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1,M4E,East Toronto,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [48]:
toronto_grouped = toronto_onehot.groupby(["PostalCode", "Borough", "Neighborhoods"]).mean().reset_index()

toronto_grouped.shape
#toronto_grouped

(39, 232)

In [106]:
indicators = ['st', 'nd', 'rd']

num_top_venues = 5

# creating columns Per the number of top venues:
areaColumns = ['PostalCode', 'Borough', 'Neighborhoods']
freqColumns = []
for ind in np.arange(num_top_venues):
    try:
        freqColumns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        freqColumns.append('{}th Most Common Venue'.format(ind+1))
columns = areaColumns+freqColumns

#### Create a new DF:

In [107]:
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['PostalCode'] = toronto_grouped['PostalCode']
neighborhoods_venues_sorted['Borough'] = toronto_grouped['Borough']
neighborhoods_venues_sorted['Neighborhoods'] = toronto_grouped['Neighborhoods']

for ind in np.arange(toronto_grouped.shape[0]):
    row_categories = toronto_grouped.iloc[ind, :].iloc[3:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    neighborhoods_venues_sorted.iloc[ind, 3:] = row_categories_sorted.index.values[0:num_top_venues]

print(neighborhoods_venues_sorted.shape)
neighborhoods_venues_sorted

(39, 8)


,PostalCode,Borough,Neighborhoods,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,M4E,East Toronto,The Beaches,Health Food Store,Neighborhood,Trail,Pub,Discount Store
1,M4K,East Toronto,"The Danforth West, Riverdale",Greek Restaurant,Italian Restaurant,Coffee Shop,Restaurant,Ice Cream Shop
2,M4L,East Toronto,"India Bazaar, The Beaches West",Park,Sandwich Place,Brewery,Steakhouse,Restaurant
3,M4M,East Toronto,Studio District,Café,Coffee Shop,Gastropub,Brewery,Bakery
4,M4N,Central Toronto,Lawrence Park,Park,Swim School,Bus Line,Yoga Studio,Deli / Bodega
5,M4P,Central Toronto,Davisville North,Department Store,Park,Gym / Fitness Center,Breakfast Spot,Sandwich Place
6,M4R,Central Toronto,"North Toronto West, Lawrence Park",Clothing Store,Sporting Goods Shop,Coffee Shop,Yoga Studio,Rental Car Location
7,M4S,Central Toronto,Davisville,Pizza Place,Dessert Shop,Sandwich Place,Coffee Shop,Sushi Restaurant
8,M4T,Central Toronto,"Moore Park, Summerhill East",Park,Trail,Yoga Studio,Dance Studio,Electronics Store
9,M4V,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest...",Coffee Shop,Pub,Café,Liquor Store,Vietnamese Restaurant


#### Set Number of Clusters:

In [108]:
kclusters = 5

In [109]:
toronto_grouped_clustering = toronto_grouped.drop(["PostalCode", "Borough", "Neighborhoods"], 1)

### Using K-Means to Cluster The Neighbourhoods:

In [110]:
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)
kmeans

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=5, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=0, tol=0.0001, verbose=0)

In [111]:
# Check the Labels of the Clusters generated for each row in the DF:
kmeans.labels_[0:10]

array([0, 3, 3, 1, 2, 3, 3, 3, 2, 3])

#### Creating a New DF that includes the cluster as well as the Top 10 venues for each Neighborhood:

In [112]:
toronto_merged = new_df_toronto.copy()
toronto_merged

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
5,M4P,Central Toronto,Davisville North,43.712751,-79.390197
6,M4R,Central Toronto,"North Toronto West, Lawrence Park",43.715383,-79.405678
7,M4S,Central Toronto,Davisville,43.704324,-79.388790
8,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160
9,M4V,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest...",43.686412,-79.400049


In [113]:
#Add Clustering Labels:
toronto_merged["Cluster Labels"] = kmeans.labels_

In [114]:
# Merge 'toronto_data' with 'toronto_grouped' to add Lat./Long. for each Neighborhood:
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.drop(["Borough", "Neighborhoods"], 1).set_index("PostalCode"), on="PostalCode")

print(toronto_merged.shape)
toronto_merged.head() 

(39, 11)


,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Health Food Store,Neighborhood,Trail,Pub,Discount Store
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,3,Greek Restaurant,Italian Restaurant,Coffee Shop,Restaurant,Ice Cream Shop
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572,3,Park,Sandwich Place,Brewery,Steakhouse,Restaurant
3,M4M,East Toronto,Studio District,43.659526,-79.340923,1,Café,Coffee Shop,Gastropub,Brewery,Bakery
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,2,Park,Swim School,Bus Line,Yoga Studio,Deli / Bodega


### Sort the Results by Cluster Labels:

In [115]:
print(toronto_merged.shape)
toronto_merged.sort_values(["Cluster Labels"], inplace=True)
toronto_merged

(39, 11)


,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Health Food Store,Neighborhood,Trail,Pub,Discount Store
34,M6P,West Toronto,"High Park, The Junction South",43.661608,-79.464763,1,Mexican Restaurant,Café,Thai Restaurant,Grocery Store,Fried Chicken Joint
33,M6K,West Toronto,"Brockton, Parkdale Village, Exhibition Place",43.636847,-79.428191,1,Café,Coffee Shop,Breakfast Spot,Bakery,Grocery Store
31,M6H,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259,1,Pharmacy,Bakery,Middle Eastern Restaurant,Music Venue,Park
30,M6G,Downtown Toronto,Christie,43.669542,-79.422564,1,Grocery Store,Café,Park,Athletics & Sports,Diner
26,M5T,Downtown Toronto,"Kensington Market, Chinatown, Grange Park",43.653206,-79.400049,1,Café,Coffee Shop,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Mexican Restaurant
25,M5S,Downtown Toronto,"University of Toronto, Harbord",43.662696,-79.400049,1,Café,Bookstore,Bar,Japanese Restaurant,Sandwich Place
38,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558,1,Skate Park,Pizza Place,Recording Studio,Fast Food Restaurant,Farmers Market
3,M4M,East Toronto,Studio District,43.659526,-79.340923,1,Café,Coffee Shop,Gastropub,Brewery,Bakery
8,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160,2,Park,Trail,Yoga Studio,Dance Studio,Electronics Store


### Create The Map:

In [116]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# Setting the Color Scheme for the Clusters:
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# Adding Markers to the Map:
markers_colors = []
for lat, lon, post, bor, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['PostalCode'], 
                                             toronto_merged['Borough'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup('{} ({}): {} - Cluster {}'.format(bor, post, poi, cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examining the Clusters:

In [117]:
# Cluster 1:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,East Toronto,0,Health Food Store,Neighborhood,Trail,Pub,Discount Store


In [118]:
# Cluster 2:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
34,West Toronto,1,Mexican Restaurant,Café,Thai Restaurant,Grocery Store,Fried Chicken Joint
33,West Toronto,1,Café,Coffee Shop,Breakfast Spot,Bakery,Grocery Store
31,West Toronto,1,Pharmacy,Bakery,Middle Eastern Restaurant,Music Venue,Park
30,Downtown Toronto,1,Grocery Store,Café,Park,Athletics & Sports,Diner
26,Downtown Toronto,1,Café,Coffee Shop,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Mexican Restaurant
25,Downtown Toronto,1,Café,Bookstore,Bar,Japanese Restaurant,Sandwich Place
38,East Toronto,1,Skate Park,Pizza Place,Recording Studio,Fast Food Restaurant,Farmers Market
3,East Toronto,1,Café,Coffee Shop,Gastropub,Brewery,Bakery


In [119]:
# Cluster 3:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
8,Central Toronto,2,Park,Trail,Yoga Studio,Dance Studio,Electronics Store
4,Central Toronto,2,Park,Swim School,Bus Line,Yoga Studio,Deli / Bodega
23,Central Toronto,2,Park,Trail,Jewelry Store,Sushi Restaurant,Yoga Studio
10,Downtown Toronto,2,Park,Playground,Trail,Yoga Studio,Dance Studio


In [120]:
# Cluster 4:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
12,Downtown Toronto,3,Coffee Shop,Sushi Restaurant,Japanese Restaurant,Gay Bar,Restaurant
36,West Toronto,3,Coffee Shop,Café,Italian Restaurant,Sushi Restaurant,Pizza Place
35,West Toronto,3,Gift Shop,Breakfast Spot,Bookstore,Italian Restaurant,Bar
1,East Toronto,3,Greek Restaurant,Italian Restaurant,Coffee Shop,Restaurant,Ice Cream Shop
2,East Toronto,3,Park,Sandwich Place,Brewery,Steakhouse,Restaurant
32,West Toronto,3,Bar,Coffee Shop,Asian Restaurant,Restaurant,Café
9,Central Toronto,3,Coffee Shop,Pub,Café,Liquor Store,Vietnamese Restaurant
29,Downtown Toronto,3,Coffee Shop,Café,Hotel,Gym,Restaurant
28,Downtown Toronto,3,Coffee Shop,Café,Restaurant,Japanese Restaurant,Italian Restaurant
27,Downtown Toronto,3,Airport Service,Airport Lounge,Airport Terminal,Airport,Sculpture Garden


In [121]:
# Cluster 5:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
22,Central Toronto,4,Music Venue,Garden,Dance Studio,Ethiopian Restaurant,Electronics Store
